# 구글 번역기 오류를 고쳐주는 프로그램(과학 논문 번역용)

In [ ]:
class Queue:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def enqueue(self, item):
        self.items.insert(0,item)

    def dequeue(self):
        return self.items.pop()

    def size(self):
        return len(self.items)



In [7]:
from googletrans import Translator

translator = Translator()
print(translator.translate('황산', src='ko', dest='en').text)

Sulfuric acid


In [2]:
e='A further air foil comprises an air deflector for the bumper of a highway vehicle.'
from googletrans import Translator

translator = Translator()
c=translator.translate(e, src='en', dest='ko').text
a={'deflector':'변류기','Deflector':'변류기'}
b=c.split()
d=e.split()
w=''
if translator.translate('deflector', src='en', dest='ko').text in b:
    for i in b:
        if i != translator.translate('deflector', src='en', dest='ko').text:
            w.append(i)
elif translator.translate('deflector', src='en', dest='ko').text not in b:
    indexcal=d.index('deflector')
    q= translator.translate(d[indexcal-1], src='en', dest='ko').text
    
    t=len(b)
    for i in b:
        while t>0:
            p=0
            if i==q and b.index(i)>p:
                p=b.index(i)
            
            t-=1
            
    b.insert(p+1,a['deflector'])
    w=b
print(w)           
    


['추가의', '변류기', '공기', '포일은', '고속도로', '차량의', '범퍼', '용', '공기', '전향', '장치를', '포함한다.']


In [1]:
from googletrans import Translator

translator = Translator()
e='A further air foil comprises an air deflector for the bumper of a highway vehicle.'
c=translator.translate(e, src='en', dest='ko').text
a={1:'Deflector',2:'전향 장치', 3:'변류기'}

print(e)
if a[1].lower() in e or a[1] in e :
    if a[2] in c :
        d = c.replace(a[2], a[3])
        print(d)
    else :
        print(c)

A further air foil comprises an air deflector for the bumper of a highway vehicle.
추가의 공기 포일은 고속도로 차량의 범퍼 용 공기 변류기를 포함한다.


In [1]:
from googletrans import Translator
import csv

#클래스를 이용하여 번역 오류를 수정
class myTranslator:
    def __init__(self, eng, kor):
        self.eng = eng
        self.kor = kor

    def Translate(self, en_word, miss_kword, fix_kword):
        if en_word.lower() in self.eng.lower() :
            if miss_kword in self.kor :
                self.kor = self.kor.replace(miss_kword, fix_kword)
        return(self.kor)

translator = Translator()

en_sentence=input('번역하고 싶은 문장을 넣어 주세요:')
kor_sentence=translator.translate(en_sentence, src='en', dest='ko').text
t = myTranslator(en_sentence, kor_sentence)

f=open('구글번역오류사전.csv')
csv_dic=csv.reader(f)
firstLine=True
for line in csv_dic :
    if(firstLine) :
        firstLine = False
        continue
        
    kor_new = t.Translate(line[0], line[1], line[2])

print('고치기 전:',kor_sentence)
print('새로 고쳐져서 번역된 문장:',kor_new)


번역하고 싶은 문장을 넣어 주세요:Amplify
고치기 전: 더욱 상 세히 하다
새로 고쳐져서 번역된 문장: 더욱 상 세히 하다


In [ ]:
'A further air foil comprises an air deflector for the bumper of a highway vehicle.'

In [7]:
## parser.py
import requests
from bs4 import BeautifulSoup

req = requests.get('https://beomi.github.io/beomi.github.io_old/')
html = req.text
soup = BeautifulSoup(html, 'html.parser')
my_titles = soup.select(
    'body > h3 > a'
    )
## my_titles는 list 객체
for title in my_titles:
    ## Tag안의 텍스트
    print(title.text)
    ## Tag의 속성을 가져오기(ex: href속성)
    #print(title.get('href'))

나만의 웹 크롤러 만들기(4): Django로 크롤링한 데이터 저장하기
나만의 웹 크롤러 만들기(3): Selenium으로 무적 크롤러 만들기
Django에 Social Login 붙이기: Django세팅부터 Facebook/Google 개발 설정까지
Django에 Custom인증 붙이기
나만의 웹 크롤러 만들기(2): Login with Session
나만의 웹 크롤러 만들기 with Requests/BeautifulSoup
Celery로 TelegramBot 알림 보내기
Virtualenv/VirtualenvWrapper OS별 설치&이용법
[DjangoTDDStudy] #02: UnitTest 이용해 기능 테스트 하기
[DjangoTDDStudy] #01: 개발환경 세팅하기(Selenium / ChromeDriver)
[DjangoTDDStudy] #00: 스터디를 시작하며
Fabric Put 커맨드가 No Such File Exception을 반환할 때 해결법
CKEditor의 라이센스와 오픈소스 라이센스
ReactNative The Basis 번역을 끝냈습니다.
[React Native 번역]#01: 시작하기
[번역] 장고(Django)와 함께하는 Celery 첫걸음
Chrome Native Adblockr 대체하기
CustoMac 설치 분투기
Ubuntu14.04에 OhMyZsh 설치
Ubuntu14.04에서 pip로 mysqlclient 설치 실패시
Ubuntu14.04에서 Python3기반 virtualenvwrapper 설치
mac OS X에서 pip virtualenvwrapper 설치 시 uninstalling six 에서 Exception 발생 시
Fabric for Python3 (Fabric3)
Windows에서 pip로 mysqlclient 설치 실패시(python3.4/3.5)
맥에서 윈도RDP로 접속시 한영전환하기.
pip로 mysqlclient설치 중 mac os x에서 egg_info / OSError 발생시 대처방법


In [ ]:
#화학식 관련 정보 출력
import os
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from googletrans import Translator

#chemword=input('화학식이나 화학식이름을 입력해 주세요:')

wolan=translator.detect(chemword).lang

ff_driver = webdriver.Firefox()
ff_driver.get("https://www.apple.com/kr/")

query = ff_driver.find_element_by_id("lst-ib").send_keys("macbook pro")
ff_driver.find_element_by_name("btnK").click()

WebDriverWait(ff_driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "h3.r")))

page_results = ff_driver.find_elements(By.CSS_SELECTOR, "h3.r")

for item in page_results:
    print(item.text)

In [9]:
chemword=input('화학식이나 화학식이름을 입력해 주세요:')

print(translator.detect(chemword).lang)

화학식이나 화학식이름을 입력해 주세요:carbon
en


In [ ]:
C:\Users\sean5\Downloads\test_images

In [4]:
from selenium import webdriver

driver = webdriver.Chrome('/Users/sean5/Downloads/test_images/chromedriver')
driver.implicitly_wait(3)
# url에 접근한다.
driver.get('https://www.wolframalpha.com/')

keyword = 'CO2'

driver.find_element_by_xpath("//*[@id=\"query\"]").send_keys(keyword)
driver.find_element_by_xpath("//*[@id=\"input\"]/fieldset/div/font/font/input").submit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="input"]/fieldset/div/font/font/input"}
  (Session info: chrome=74.0.3729.169)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Windows NT 10.0.17134 x86_64)


In [5]:

driver.find_element_by_xpath("//*[@id=\"input\"]/fieldset/div/font/font/input").click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="input"]/fieldset/div/font/font/input"}
  (Session info: chrome=74.0.3729.169)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Windows NT 10.0.17134 x86_64)
